In [1]:
from tensorflow.keras.models import load_model

model_path = "resnet_model_3_aug"
model = load_model(model_path)

ValueError: File format not supported: filepath=resnet_model_3_aug. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(resnet_model_3_aug, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
from custom_transformers import SquareCutout, CircleCutout

In [ ]:
from torchvision import transforms as T
import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.resnet50 import preprocess_input
import cv2
import numpy as np
from PIL import Image
import random, os,torch

In [ ]:
def seed_everything(seed: int=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(42)

In [ ]:
cifar10 = tf.keras.datasets.cifar10

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
print(f"X_train.shape: {X_train.shape} \nY_train.shape: {Y_train.shape} \nX_test.shape: {X_test.shape} \nY_test.shape: {Y_test.shape}")


X_train.shape: (50000, 32, 32, 3) 
Y_train.shape: (50000, 1) 
X_test.shape: (10000, 32, 32, 3) 
Y_test.shape: (10000, 1)


In [ ]:

class AugmentedCIFAR10ResNetGenerator(Sequence):
    def __init__(self, x_data, y_data, batch_size=32, shuffle=True, augmenter=None):
        self.x = x_data
        self.y = y_data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmenter = augmenter
        self.indices = np.arange(len(self.x))

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x = self.x[batch_indices]
        batch_y = self.y[batch_indices]

        #augmentation using selected augmenter
        batch_x_aug = np.array([self.augmenter(img, label)[0] for img, label in zip(batch_x, batch_y)])

        #resize each image to 224x224 using OpenCV
        batch_x_resized = np.array([cv2.resize(img, (224, 224)) for img in batch_x_aug])

        #preprocess for ResNet50
        batch_x_preprocessed = preprocess_input(batch_x_resized.astype(np.float32))

        return batch_x_preprocessed, batch_y



In [ ]:
Y_train = Y_train.squeeze()
Y_test = Y_test.squeeze()

In [ ]:
num_samples = len(X_train)
subset_size = num_samples // 10  #10% of training data

indices = np.random.permutation(num_samples)
subset_indices = indices[:subset_size]

X_subset = X_train[subset_indices]
Y_subset = Y_train[subset_indices]

In [ ]:
augmenter = SquareCutout(size=16, color=False)
augmented_train_gen = AugmentedCIFAR10ResNetGenerator(X_subset, Y_subset, batch_size=32, augmenter=augmenter)